# Introduction to posless (bxibase > 6.x)

Version 6.x of bxi.base.parserconf supports command-based configuration files. That is, a configuration file can be specific to a given command. When launching a new command say `bxi-foo`, a configuration file for that command is looked for. The location file depends on various parameters described below:

0. If a file is specified with the option `--config-file`, it is directly used.
1. Otherwise, if a specific directory is specified with option `--config-dir`, it is used to find configuration files according to the policy defined below. Else, if the user is root, files are searched in `/etc`, otherwise, they are searched in `~/.config/`. 
2. Unless a specific directory is specified with option `--config-dir`, the `config_dirname` passed to `parserconf.addargs()`, is appended to the path. It defaults to `bxi`. Therefore, files are searched in `/etc/bxi/` when launched as root, or `~/.config/bxi` when launched as a normal user by default.
3. Then if a file named `bxi-foo.conf` is found, it is used for the configuration of that `bxi-foo` command.
4. Otherwise, if a `domain_name` say `bar` has been given to the `parserconf.addargs()`, and if a file named `bar.conf` exists in that path, it is used for that `bxi-foo` command.
5. Otherwise, if a file named `default.conf` is found, it is used for that `bxi-foo` command.

The `bxibase` package provides the `bxilog-posless.py` command in the documentation examples directory.
Its content is rather simple:

In [12]:
%%bash

cat ../../../packaged/doc/examples/bxilog-posless

#!/usr/bin/env python

import sys
import bxi.base.log as bxilog
import bxi.base.posless as posless
import bxi.base.parserconf as bxiparserconf

if __name__ == '__main__':
    parser = posless.ArgumentParser(description='BXI Log Posless Example',
                                   formatter_class=bxiparserconf.FilteredHelpFormatter)

    # Do it first!
    bxiparserconf.addargs(parser, domain_name='bar')

    default = bxiparserconf.getdefaultvalue(parser, ['My Section'], 'variable',
                                            bxilog.getLogger('config'),
                                            'Default_Value',
                                            None,
                                            )
    parser.add_argument('--variable', default=default,
                        envvar='VAR',
                        help="Do something with variable. Current: %(default)s.")
    default = bxiparserconf.getdefaultvalue(parser, ['My Section'], 'stuff',
                               

First let's have a look at its usage:

In [13]:
%%bash 

../../../packaged/doc/examples/bxilog-posless --help

usage: bxilog-posless [-h] [--config-dir DIR] [-C FILE] [--help-logs] [--loglevels] [--output-default-logcfg]
                      [--logcfgfile logcfgfile] [-l log-console-filters] [--log-file-filters log-file-filters]
                      [--log-file-path PATH] [--help-full] [--variable VARIABLE] [--stuff STUFF]

BXI Log Posless Example

optional arguments:
  -h, --help           show this help message and exit
  --help-logs          show detailed logging options and exit
  --help-full          show all options and exit
  --variable VARIABLE  Do something with variable. Current: Default_Value.
  --stuff STUFF        Do something with stuff. Current: Default_stuff.

File Based Configuration:
  --config-dir DIR     The directory where the configuration file must be looked for. Value: /home/vigneras/.config/bxi.
                       Environment variable: BXICONFIGDIR


Three things must be noticed:

1. it uses a configuration directory (`--config-dir`): this is provided by parserconf module; most command will provide this option. This is where common variables used by multiple commands can be factorized. More on that soon.
2. It provides a `--variable` option. This is just an example. We will see how its content depends on the configuration file.
3. It also provides logging configuration `--help-logs`. More on that later.

Executing this program, gives:

In [14]:
!../../../packaged/doc/examples/bxilog-posless

Value of variable: Default_Value
Value of stuff: Default_stuff
This process just starts, arguments: ['../../../packaged/doc/examples/bxilog-posless']
Starting
[E] one.logg Something wrong happened?
[C] another. And here, something even worse!
This process ends normally


# Command Line Variables Configuration

When the bxiparserconf is used, a configuration variable can be given its value according to the following increasing policy:

1. a default value;
1. a configuration file;
2. an environment variable;
3. from the command line.

The default value is given in the code itself when nothing else has been found. However if a configuration file is found, the variable is taken from it. In our example, if we create a default configuration file:

In [15]:
import os
import tempfile

DIR=tempfile.mkdtemp()

fileconf = os.path.join(DIR, 'default.conf')

with open(fileconf, 'w') as f:
    f.write("""
[My Section]
    variable = 'foo'
""")

print("File %s created" % fileconf)

File /tmp/.private/vigneras/tmpLgHWXh/default.conf created


In [16]:
!../../../packaged/doc/examples/bxilog-posless --config-dir {DIR} --help

usage: bxilog-posless [-h] [--config-dir DIR] [-C FILE] [--help-logs] [--loglevels] [--output-default-logcfg]
                      [--logcfgfile logcfgfile] [-l log-console-filters] [--log-file-filters log-file-filters]
                      [--log-file-path PATH] [--help-full] [--variable VARIABLE] [--stuff STUFF]

BXI Log Posless Example

optional arguments:
  -h, --help           show this help message and exit
  --help-logs          show detailed logging options and exit
  --help-full          show all options and exit
  --variable VARIABLE  Do something with variable. Current: foo.
  --stuff STUFF        Do something with stuff. Current: Default_stuff.

File Based Configuration:
  --config-dir DIR     The directory where the configuration file must be looked for. Value: /tmp/.private/vigneras/tmpLgHWXh.
                       Environment variable: BXICONFIGDIR


Since the `default.conf` configuration file has been found, the value defined for `variable` is used and shown in the help. However, this value can be overridden by a new configuration file. Since the domain of the `bxilog-posless` command is specified (in the code) to be `bar`, the following configuration file will be used instead:

In [17]:
fileconf = os.path.join(DIR, 'bar.conf')

with open(fileconf, 'w') as f:
    f.write("""
[My Section]
    variable = 'bar'
""")

print("File %s created" % fileconf)
    
!../../../packaged/doc/examples/bxilog-posless --config-dir {DIR} --help

File /tmp/.private/vigneras/tmpLgHWXh/bar.conf created
usage: bxilog-posless [-h] [--config-dir DIR] [-C FILE] [--help-logs] [--loglevels] [--output-default-logcfg]
                      [--logcfgfile logcfgfile] [-l log-console-filters] [--log-file-filters log-file-filters]
                      [--log-file-path PATH] [--help-full] [--variable VARIABLE] [--stuff STUFF]

BXI Log Posless Example

optional arguments:
  -h, --help           show this help message and exit
  --help-logs          show detailed logging options and exit
  --help-full          show all options and exit
  --variable VARIABLE  Do something with variable. Current: bar.
  --stuff STUFF        Do something with stuff. Current: Default_stuff.

File Based Configuration:
  --config-dir DIR     The directory where the configuration file must be looked for. Value: /tmp/.private/vigneras/tmpLgHWXh.
                       Environment variable: BXICONFIGDIR


Finally, one step further is to create a configuration with named `bxilog-posless.conf` with a specific configuration that overrides previous ones:

In [18]:
fileconf = os.path.join(DIR, 'bxilog-posless.conf')

with open(fileconf, 'w') as f:
    f.write("""
[My Section]
    variable = 'baz'
""")

print("File %s created" % fileconf)
    
!../../../packaged/doc/examples/bxilog-posless --config-dir {DIR} --help

File /tmp/.private/vigneras/tmpLgHWXh/bxilog-posless.conf created
usage: bxilog-posless [-h] [--config-dir DIR] [-C FILE] [--help-logs] [--loglevels] [--output-default-logcfg]
                      [--logcfgfile logcfgfile] [-l log-console-filters] [--log-file-filters log-file-filters]
                      [--log-file-path PATH] [--help-full] [--variable VARIABLE] [--stuff STUFF]

BXI Log Posless Example

optional arguments:
  -h, --help           show this help message and exit
  --help-logs          show detailed logging options and exit
  --help-full          show all options and exit
  --variable VARIABLE  Do something with variable. Current: baz.
  --stuff STUFF        Do something with stuff. Current: Default_stuff.

File Based Configuration:
  --config-dir DIR     The directory where the configuration file must be looked for. Value: /tmp/.private/vigneras/tmpLgHWXh.
                       Environment variable: BXICONFIGDIR


As we have seen, a specific configuration directory can be specified. A specific configuration file can also be specified. In this case, it is used directly:

In [19]:
fileconf = os.path.join(DIR, 'specific.conf')

with open(fileconf, 'w') as f:
    f.write("""
[My Section]
    variable = 'spec'
""")

print("File %s created" % fileconf)
    
!../../../packaged/doc/examples/bxilog-posless --config-file {fileconf} --help

File /tmp/.private/vigneras/tmpLgHWXh/specific.conf created
usage: bxilog-posless [-h] [--config-dir DIR] [-C FILE] [--help-logs] [--loglevels] [--output-default-logcfg]
                      [--logcfgfile logcfgfile] [-l log-console-filters] [--log-file-filters log-file-filters]
                      [--log-file-path PATH] [--help-full] [--variable VARIABLE] [--stuff STUFF]

BXI Log Posless Example

optional arguments:
  -h, --help           show this help message and exit
  --help-logs          show detailed logging options and exit
  --help-full          show all options and exit
  --variable VARIABLE  Do something with variable. Current: spec.
  --stuff STUFF        Do something with stuff. Current: Default_stuff.

File Based Configuration:
  --config-dir DIR     The directory where the configuration file must be looked for. Value: None. Environment variable:
                       BXICONFIGDIR


Of course the environment variable is more specific that configuration files:

In [20]:
%env VAR='env'

!../../../packaged/doc/examples/bxilog-posless --config-dir {DIR} --help

env: VAR='env'
usage: bxilog-posless [-h] [--config-dir DIR] [-C FILE] [--help-logs] [--loglevels] [--output-default-logcfg]
                      [--logcfgfile logcfgfile] [-l log-console-filters] [--log-file-filters log-file-filters]
                      [--log-file-path PATH] [--help-full] [--variable VARIABLE] [--stuff STUFF]

BXI Log Posless Example

optional arguments:
  -h, --help           show this help message and exit
  --help-logs          show detailed logging options and exit
  --help-full          show all options and exit
  --variable VARIABLE  Do something with variable. Current: 'env'.
  --stuff STUFF        Do something with stuff. Current: Default_stuff.

File Based Configuration:
  --config-dir DIR     The directory where the configuration file must be looked for. Value: /tmp/.private/vigneras/tmpLgHWXh.
                       Environment variable: BXICONFIGDIR


And finally, the command line is even more specific:

In [21]:
!../../../packaged/doc/examples/bxilog-posless --config-dir {DIR} --variable=abc --help

usage: bxilog-posless [-h] [--config-dir DIR] [-C FILE] [--help-logs] [--loglevels] [--output-default-logcfg]
                      [--logcfgfile logcfgfile] [-l log-console-filters] [--log-file-filters log-file-filters]
                      [--log-file-path PATH] [--help-full] [--variable VARIABLE] [--stuff STUFF]

BXI Log Posless Example

optional arguments:
  -h, --help           show this help message and exit
  --help-logs          show detailed logging options and exit
  --help-full          show all options and exit
  --variable VARIABLE  Do something with variable. Current: abc.
  --stuff STUFF        Do something with stuff. Current: Default_stuff.

File Based Configuration:
  --config-dir DIR     The directory where the configuration file must be looked for. Value: /tmp/.private/vigneras/tmpLgHWXh.
                       Environment variable: BXICONFIGDIR


# Configuration Files Inclusion

The `bxi.base.parserconf` module implements configuration file inclusion: one file can include another. When file `F` includes file `I`, all variables defined in `I` are also defined in `F`, and their values are the one in `I` unless they are overridden by `F`. Consider the following example:

In [23]:
fileconf = os.path.join(DIR, 'I.conf')

with open(fileconf, 'w') as f:
    f.write("""
[My Section]
    variable = 'defined_in_I'
    stuff = 'defined_in_I'
""")

print("File %s created" % fileconf)

fileconf = os.path.join(DIR, 'F.conf')

with open(fileconf, 'w') as f:
    f.write("""
includes = I.conf,
[My Section]
    stuff = 'defined_in_F'
""")

print("File %s created" % fileconf)

    
!../../../packaged/doc/examples/bxilog-posless --config-file {fileconf} --help

File /tmp/.private/vigneras/tmpLgHWXh/I.conf created
File /tmp/.private/vigneras/tmpLgHWXh/F.conf created
usage: bxilog-posless [-h] [--config-dir DIR] [-C FILE] [--help-logs] [--loglevels] [--output-default-logcfg]
                      [--logcfgfile logcfgfile] [-l log-console-filters] [--log-file-filters log-file-filters]
                      [--log-file-path PATH] [--help-full] [--variable VARIABLE] [--stuff STUFF]

BXI Log Posless Example

optional arguments:
  -h, --help           show this help message and exit
  --help-logs          show detailed logging options and exit
  --help-full          show all options and exit
  --variable VARIABLE  Do something with variable. Current: 'env'.
  --stuff STUFF        Do something with stuff. Current: defined_in_F.

File Based Configuration:
  --config-dir DIR     The directory where the configuration file must be looked for. Value: None. Environment variable:
                       BXICONFIGDIR


The current value of 'stuff' is 'defined_in_F' since it overwrites the included value from `I.conf`.

# Logging Configuration Files

The `bxi.base.parserconf` module implements a special policy for logging configuration: it looks after a variable called `bxilog.default.configfile` and if found, use it as a path towards the logging configuration file. If not found however, a default logging configuration is used. This default logging configuration can be output on `stdout` using the option `--output-default-logcfg` as in the following example:

In [24]:
!../../../packaged/doc/examples/bxilog-posless --output-default-logcfg

setsighandler = True
handlers = console, file
[console]
    colors = 216_dark
    stderr_level = WARNING
    filters = :output
    module = bxi.base.log.console_handler
[file]
    path = /tmp/.private/vigneras/%(prog)s.bxilog
    append = True
    filters = auto
    module = bxi.base.log.file_handler


As you see, by default, two log handlers are defined:

- console handler: output on the standard output and standard error;
- file handler: output to a file.

Let's save this content to a file in our temporary directory:

In [26]:
!../../../packaged/doc/examples/bxilog-posless --output-default-logcfg > {DIR}/default.bxilog.conf

Let's then modify its content to lower the filtering:

In [27]:
!sed -i "s/:output/:debug/" {DIR}/default.bxilog.conf
!cat {DIR}/default.bxilog.conf

setsighandler = True
handlers = console, file
[console]
    colors = 216_dark
    stderr_level = WARNING
    filters = :debug
    module = bxi.base.log.console_handler
[file]
    path = /tmp/.private/vigneras/%(prog)s.bxilog
    append = True
    filters = auto
    module = bxi.base.log.file_handler


Let's modify the configuration file `I.conf` so it references this logging configuration file:

In [30]:
!echo -e '[Defaults]\n\tbxilog.default.configfile = default.bxilog.conf' >> {DIR}/I.conf
!cat {DIR}/I.conf


[My Section]
    variable = 'defined_in_I'
    stuff = 'defined_in_I'
bxilog.default.configfile = default.bxilog.conf
[Defaults]
	bxilog.default.configfile = default.bxilog.conf


In [31]:
!../../../packaged/doc/examples/bxilog-posless --config-file {DIR}/F.conf

[D] ~bxilog  Initialization done
[D] ~bxilog. Setting signal handler process wide
[D] ~bxilog. Signal handler set for 11: Segmentation fault
[D] ~bxilog. Signal handler set for 7: Bus error
[D] ~bxilog. Signal handler set for 8: Floating point exception
[D] ~bxilog. Signal handler set for 4: Illegal instruction
[D] ~bxilog. Signal handler set for 2: Interrupt
[D] ~bxilog. Signal handler set for 15: Terminated
[I] ~bxilog. Signal handlers set
[D] bxilog-p BXI logging configuration: {'setsighandler': 'True', 'handlers': ['console', 'file'], 'console': {'colors': '216_dark', 'stderr_level': 'WARNING', 'filters': ':debug', 'module': 'bxi.base.log.console_handler'}, 'file': {'path': '/tmp/.private/vigneras/bxilog-posless.bxilog', 'append': 'True', 'filters': 'auto', 'module': 'bxi.base.log.file_handler'}}
[D] config   Return defined_in_I for value variable in section ['My Section']
[D] config   Return defined_in_F for value stuff in section ['My Section']
Value of variable: 'env'
Value of s

The level of the console handler has changed according to the logging configuration file defined in file `I.conf` which is included by `F.conf`.

# Advanced Usage

Commands can be grouped by big domains in such a way that configuration variables can be factorized.